# 动机
获取到特征组后，同一特征组中的数据较为稠密，比较适合使用课程学习的算法进行进一步的分析。在这里我们使用试试PCA降维和聚类，看看能否发现一些比较有趣的Pattern。

## PCA
部分特征组的特征数还是较多，做PCA有利于数据降维，也便于通过可视化来观察降维后的数据特点。

这里的绘图是弹出窗口式的，为了便于通过鼠标拖拽三维散点图。坏处是无法把图片保存到notebook。

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib

# Load data from a pickle file
try:
    data : pd = pd.read_pickle('../data/data_remove_dup.pkl')
except:
    print("未在路径中找到目标数据表，先运行其他脚本")
    raise
data_original = data.copy()
data = data.iloc[:, 1:].drop(columns=["病床号"])
data = data.groupby("住院号码").mean(numeric_only=True).reset_index()

matplotlib.rcParams["font.sans-serif"] = ["SimHei"]  # 指定默认字体
matplotlib.rcParams["axes.unicode_minus"] = False

import matplotlib.pyplot as plt

# Select the specified columns
groups = {
    '餐后血糖调节': ('C肽（餐后2小时）', '胰岛素（餐后2小时）', '葡萄糖(餐后2小时)'),
    '空腹胰岛素分泌': ('C肽', '胰岛素'),
    '糖尿病监测': ('糖化血红蛋白',),
    '血脂和基础代谢评估': ('低密度脂蛋白', '总胆固醇', '甘油三酯', '磷', '糖化白蛋白', '葡萄糖', '镁', '高密度脂蛋白'),
    '肾功能及电解质评估': ('尿素', '尿酸', '氯', '肌酐', '钙', '钠', '钾'),
    '肾功能及钙磷代谢评估': ('尿肌酐', '尿钙', '24小时尿磷'),
    '骨形成与甲状腺排查': ('β-胶原特殊序列', '促甲状腺素受体抗体', '总I型胶原氨基端延长肽', '骨钙素(N-MID)'),
    '血钙-骨代谢调节评估': ('25-羟基维生素D', '甲状旁腺激素', '降钙素'),
    '骨形成标志物': ('骨源碱性磷酸酶',),
    '性激素水平': ('促卵泡成熟素', '促黄体生成素', '叶酸', '孕酮', '泌乳素', '睾酮', '硫酸去氢表雄酮', '雌二醇'),
    '甲状腺功能评估': ('促甲状腺素', '反三碘甲状腺原氨酸', '总三碘甲状腺原氨酸', '总四碘甲状腺原氨酸', 
                  '游离三碘甲状腺原氨酸', '游离甲状腺素', '甲状腺球蛋白', '甲状腺过氧化物酶抗体'),
    '炎症因子监测': ('干扰素γ', '白介素10', '白介素17A', '白介素1β', '白介素2', '白介素2受体', 
               '白介素4', '白介素5', '白介素6', '白介素8', '肿瘤坏死因子α'),
    '肝酶及胆红素代谢': ('γ-谷氨酰转肽酶', '天门冬氨酸转氨酶', '总胆红素', '直接胆红素', '碱性磷酸酶')
}
# 对所有 groups 中的列分别做PCA，并绘图展示
for idx, (group_name, group) in enumerate(groups.items()):
    # 过滤出当前 group 中实际存在于 data 的列
    valid_columns = [col for col in group if col in data.columns]
    if not valid_columns:
        print(f"Group {idx+1} 没有有效的列，跳过。")
        continue
    group_data = data[valid_columns].dropna()
    if group_data.shape[0] < 3 or group_data.shape[1] < 2:
        print(f"Group {idx+1} 数据量不足，跳过。")
        continue
    # 归一化
    scaler = MinMaxScaler()
    # scaler = StandardScaler()
    group_normalized = scaler.fit_transform(group_data)

    group_pca = PCA()
    group_pca_result = group_pca.fit_transform(group_normalized)
    # 解释方差
    print(f"Group {idx+1} ({valid_columns})\n explained variance ratio:", group_pca.explained_variance_ratio_)
    # 绘图（弹出窗口，支持交互旋转）
    from mpl_toolkits.mplot3d import Axes3D  # 确保3D支持
    matplotlib.use('tkagg')  # 使用tkagg后端弹窗

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(
        group_pca_result[:, 0],
        group_pca_result[:, 1],
        group_pca_result[:, 2] if group_pca_result.shape[1] > 2 else [0]*group_pca_result.shape[0],
        alpha=0.7
    )
    # 自动换行标题
    title = f"PCA Result - {group_name}"
    ax.set_title(title)
    ax.set_xlabel("PC1")
    ax.set_ylabel("PC2")
    ax.set_zlabel("PC3")
plt.show(block=True)

## 聚类
我们认为，性激素、甲状腺、血脂与基础代谢三个组别呈现了比较有意思的模式。请看我们PPT演示文稿中的具体说明。下面是对 性激素、甲状腺 这两个特征组的聚类。

### 对性激素特征组聚类

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import numpy as np

sex_hormone_cols = [col for col in groups['性激素水平'] if col in data.columns]
sex_hormone_data = data.dropna(subset=sex_hormone_cols)[sex_hormone_cols]

sex_hormone_norm = scaler.fit_transform(sex_hormone_data)

pca_vis = PCA(n_components=3)
sex_hormone_pca = pca_vis.fit_transform(sex_hormone_norm)

agg = AgglomerativeClustering(n_clusters=2, linkage='ward')
labels = agg.fit_predict(sex_hormone_pca)


from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(
    sex_hormone_pca[:, 0],
    sex_hormone_pca[:, 1],
    sex_hormone_pca[:, 2],
    c=labels, cmap='coolwarm', alpha=0.7,
    label=labels
)
ax.set_title("性激素水平聚类可视化")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")

# 添加图例
import matplotlib.patches as mpatches
handles = [
    mpatches.Patch(color=plt.cm.coolwarm(0.), label='Cluster 0'),
    mpatches.Patch(color=plt.cm.coolwarm(1.), label='Cluster 1')
]
ax.legend(handles=handles, loc='best')

plt.show(block=True)

# 各类各取50个样本，反查原表“姓名”
# 先找到聚类后对应的原data索引
sex_hormone_idx = sex_hormone_data.index
# 随机选取各类50个样本索引
class0_all_idx = np.where(labels == 0)[0]
class1_all_idx = np.where(labels == 1)[0]
class0_idx = sex_hormone_idx[np.random.choice(class0_all_idx, size=min(50, len(class0_all_idx)), replace=False)]
class1_idx = sex_hormone_idx[np.random.choice(class1_all_idx, size=min(50, len(class1_all_idx)), replace=False)]


data_with_name = data_original.copy()


# 按住院号码分组取均值，保留“姓名”
data_with_name = data_with_name.iloc[:, 1:]
data_with_name = data_with_name.drop_duplicates(subset=["住院号码"])
data_with_name_grouped = data_with_name.groupby("住院号码").first().reset_index()

# 找到聚类样本的住院号码
class0_住院号码 = data.loc[class0_idx, "住院号码"]
class1_住院号码 = data.loc[class1_idx, "住院号码"]

# 反查姓名
class0_names = data_with_name_grouped[data_with_name_grouped["住院号码"].isin(class0_住院号码)]["病人姓名"].tolist()
class1_names = data_with_name_grouped[data_with_name_grouped["住院号码"].isin(class1_住院号码)]["病人姓名"].tolist()

print("聚类类别0样本姓名：", class0_names)
print("聚类类别1样本姓名：", class1_names)

### 对甲状腺特征组聚类

In [ ]:
# 选择甲状腺功能评估特征组的有效列
thyroid_cols = [col for col in groups['甲状腺功能评估'] if col in data.columns]
thyroid_data = data.dropna(subset=thyroid_cols)[thyroid_cols]

# 归一化
thyroid_norm = scaler.fit_transform(thyroid_data)

# PCA降维到3维用于可视化
thyroid_pca = pca_vis.fit_transform(thyroid_norm)

# 层次聚类
agg_thyroid = KMeans(n_clusters=2)
thyroid_labels = agg_thyroid.fit_predict(thyroid_pca)

# 可视化
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(
    thyroid_pca[:, 0],
    thyroid_pca[:, 1],
    thyroid_pca[:, 2],
    c=thyroid_labels, cmap='coolwarm', alpha=0.7
)
ax.set_title("甲状腺功能评估聚类可视化")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
handles = [
    mpatches.Patch(color=plt.cm.coolwarm(0.), label='Cluster 0'),
    mpatches.Patch(color=plt.cm.coolwarm(1.), label='Cluster 1')
]
ax.legend(handles=handles, loc='best')
plt.show(block=True)

# 各类各取50个样本，反查原表“姓名”
thyroid_idx = thyroid_data.index
class0_all_idx = np.where(thyroid_labels == 0)[0]
class1_all_idx = np.where(thyroid_labels == 1)[0]
class0_idx = thyroid_idx[np.random.choice(class0_all_idx, size=min(50, len(class0_all_idx)), replace=False)]
class1_idx = thyroid_idx[np.random.choice(class1_all_idx, size=min(50, len(class1_all_idx)), replace=False)]

# 找到聚类样本的住院号码
class0_住院号码 = data.loc[class0_idx, "住院号码"]
class1_住院号码 = data.loc[class1_idx, "住院号码"]

# 反查姓名
class0_names = data_with_name_grouped[data_with_name_grouped["住院号码"].isin(class0_住院号码)]["病人姓名"].tolist()
class1_names = data_with_name_grouped[data_with_name_grouped["住院号码"].isin(class1_住院号码)]["病人姓名"].tolist()

print("甲状腺聚类类别0样本姓名：", class0_names)
print("甲状腺聚类类别1样本姓名：", class1_names)